In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from collections import Counter
import optuna


In [2]:
train = pd.read_csv("train.csv",index_col=0)
test = pd.read_csv("test.csv",index_col=0)

train["created_at"] = pd.to_datetime(train["created_at"],format='%Y-%m-%d')
test["created_at"] = pd.to_datetime(test["created_at"],format='%Y-%m-%d')

train["month"] = train["created_at"].dt.month
train["year"] = train["created_at"].dt.year

test["month"] = test["created_at"].dt.month
test["year"] = test["created_at"].dt.year

print(train['health'].tolist().count(0))
print(train['health'].tolist().count(1))
print(train['health'].tolist().count(2))

3535
15751
698


In [3]:
#zip_cityについてはspc_commonと役割が似ていると思い、使用しませんでした。EDA(探索的データ分析)について技術を高め、具体的根拠を持って省けるよう鍛錬していきたい。
dict1 = {}#problems
dict2 = {}#zip_city
dict3 = {}#spc_common
dict4 = {}#nta
def cheak_mean(name:'variables'):
    
    name_key = set(train[name].values)
    name_list = [*name_key,]

    for list in name_list:
        dataframe = train[train[name] == list]
        health = dataframe["health"].values
        np.putmask(health, health == 2, -1)

        mean = np.mean(health)
        var = np.var(health)
        
        if name == 'problems':
            if mean > 0.9:
                dict1[list] = 0;
            elif mean>0.82:
                dict1[list] = 1;
            elif mean>0.7:
                dict1[list] = 2;
            elif mean>0.6:
                dict1[list] =3;
            elif mean>0.5:
                dict1[list] =4;
            elif mean>0.4:
                dict1[list] =5;
            elif mean>0.3:
                dict1[list] =6;
            elif mean>0.2:
                dict1[list] =7;
            elif mean>0.1:
                dict1[list] =8;
        elif name=='zip_city':
            if mean>0.8:
                dict2[list] = 0;
            elif mean>0.77:
                dict2[list] =1;
            elif mean>0.74:
                dict2[list] =2;
            elif mean>0.71:
                dict2[list] =3;
            elif mean>0.68:
                dict2[list] =4;
            elif mean>0.65:
                dict2[list] =5;
            elif mean>0.62:
                dict2[list] =6;
            elif mean>0.59:
                dict2[list] =7;
            elif mean>0.56:
                dict2[list] =8;
        elif name=='spc_common':
            if mean>0.7:
                dict3[list] = 0;
            elif mean>0.6:
                dict3[list] =1;
            elif mean>0.5:
                dict3[list] =2;
            elif mean>0.4:
                dict3[list] =3;
            elif mean>0.3:
                dict3[list] =4;
            elif mean>0.2:
                dict3[list] =5;
            elif mean>0.1:
                dict3[list] =6;
        elif name == 'nta':
            if mean>0.9:
                dict4[list]=0
            elif mean>0.8:
                dict4[list]=1
            elif mean>0.7:
                dict4[list]=2
            elif mean>0.6:
                dict4[list]=3;
            elif mean>0.5:
                dict4[list]=4;
            elif mean>0.4:
                dict4[list]=5;
            elif mean>0.3:
                dict4[list]=6;
            elif mean>0.2:
                dict4[list]=7;
            else:
                dict4[list]=8;

cheak_mean("problems")
#cheak_mean("zip_city")
cheak_mean("spc_common")
cheak_mean("nta")

C:\Users\kirin\.virtualenvs\2023winter_SMBC_green_data_challenge-PlSiMd8v\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\kirin\.virtualenvs\2023winter_SMBC_green_data_challenge-PlSiMd8v\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\kirin\.virtualenvs\2023winter_SMBC_green_data_challenge-PlSiMd8v\Lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\kirin\.virtualenvs\2023winter_SMBC_green_data_challenge-PlSiMd8v\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\kirin\.virtualenvs\2023winter_SMBC_green_data_challenge-PlSiMd8v\Lib\site-packages\numpy\core

In [5]:
train_x = train.drop(['health','zip_city','curb_loc','user_type','borocode','cb_num','nta_name','boroname','boro_ct','spc_latin','created_at','st_senate','st_assem','cncldist'],axis=1)
train_y = train['health']

test_x = test.drop(['nta_name','zip_city','curb_loc','user_type','borocode','cb_num','boroname','boro_ct','spc_latin','created_at','st_senate','st_assem','cncldist'],axis=1)
print(len(set(train['nta'].to_list())))
print(len(set(train['zip_city'].to_list())))


187
45


In [6]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(train_x.columns.to_list())


(19984, 9)
(19984,)
(19702, 9)
['tree_dbh', 'steward', 'guards', 'sidewalk', 'problems', 'spc_common', 'nta', 'month', 'year']


In [7]:
train_x[:5]

,tree_dbh,steward,guards,sidewalk,problems,spc_common,nta,month,year
0,14,NaN,NaN,Damage,NaN,English oak,QN45,6,2015
1,5,3or4,Helpful,NoDamage,NaN,crimson king maple,BX05,9,2016
2,26,NaN,NaN,NoDamage,StonesBranchLights,English oak,SI01,9,2015
3,15,NaN,NaN,Damage,NaN,honeylocust,SI11,5,2016
4,23,NaN,NaN,NoDamage,Stones,London planetree,MN03,6,2016


In [8]:
train_x['problems'] = train_x['problems'].map(dict1)
test_x['problems'] = test_x['problems'].map(dict1)
"""
train_x['zip_city'] = train_x['zip_city'].map(dict2)
test_x['zip_city'] = test_x["zip_city"].map(dict2)
"""
train_x['spc_common'] = train_x['spc_common'].map(dict3)
test_x['spc_common'] = test_x["spc_common"].map(dict3)
train_x['nta'] = train_x['nta'].map(dict4)
test_x['nta'] = test_x['nta'].map(dict4)
train_x1 = pd.get_dummies(train_x,dtype=int)

train_x1[:5]
#text_xのget_dummyはtrain_x1のcolumnとtext_xのcolumnを合わせてから行う->[8]

,tree_dbh,problems,spc_common,nta,month,year,steward_1or2,steward_3or4,steward_4orMore,guards_Harmful,guards_Helpful,guards_Unsure,sidewalk_Damage,sidewalk_NoDamage
0,14,NaN,1.0,2,6,2015,0,0,0,0,0,0,1,0
1,5,NaN,0.0,2,9,2016,0,1,0,0,1,0,0,1
2,26,2.0,1.0,2,9,2015,0,0,0,0,0,0,0,1
3,15,NaN,0.0,2,5,2016,0,0,0,0,0,0,1,0
4,23,2.0,0.0,2,6,2016,0,0,0,0,0,0,0,1


In [9]:
#xgboostはDataFrameのcolumnsの順番も一致させないと動かない模様。
test_x1 = pd.get_dummies(test_x,dtype=int)

"""
自分が行った前処理では下のコードは何も変化は起こしませんが、前処理なしで
get_dummiesを行うとtestデータには特定の要素がなく説明変数の数が異なり
xgboostが使えないことが生じた。それを防ぐためのコードがこの1行の意味です。
"""
test_x1 = test_x1.reindex(columns=train_x1.columns,index=test_x1.index,fill_value=0)

test_x1[:5]


,tree_dbh,problems,spc_common,nta,month,year,steward_1or2,steward_3or4,steward_4orMore,guards_Harmful,guards_Helpful,guards_Unsure,sidewalk_Damage,sidewalk_NoDamage
19984,15,NaN,0.0,2,9,2015,0,0,0,0,0,0,1,0
19985,5,NaN,0.0,3,10,2015,1,0,0,0,0,0,0,1
19986,4,NaN,0.0,1,8,2016,0,0,0,0,0,1,0,1
19987,7,NaN,0.0,1,8,2015,0,0,0,0,0,0,0,1
19988,6,2.0,0.0,2,10,2015,1,0,0,0,1,0,0,1


In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,f1_score
from xgboost import XGBClassifier 
from sklearn.utils.class_weight import compute_sample_weight



In [124]:
#使用するモデル部分。下コードのoprtunaによる最適化を先に実施してパラメータを決めています。
kf = StratifiedKFold(n_splits=4,shuffle=True,random_state=7777)
model = XGBClassifier(
    booster='gbtree',
    random_state=111,
    objective='multi:softprob',
    num_class=3,
    n_estimators=200,
    max_depth=2,
    learning_rate=0.065,
    eval_metric='aucpr',
    early_stopping_rounds=10,
    min_child_weight=2.265683041345576,
    gamma=3.939376582373192,
    reg_lambda=7000,
    reg_alpha=1 
)


f1_va_score = []
y_prednumber = np.zeros((len(test_x1),3))
for tr_idx,va_idx in kf.split(train_x1,train_y):
    tr_x,va_x = train_x1.iloc[tr_idx],train_x1.iloc[va_idx]
    tr_y,va_y = train_y.iloc[tr_idx],train_y.iloc[va_idx]
    class_weights = compute_sample_weight({0:3.9553512894515057, 1:1,2:14.557580573218711}, y=tr_y)
    model.fit(tr_x,tr_y,
              sample_weight = class_weights,
              eval_set=[[va_x,va_y]],
              verbose=0
             )
    #validationに対する評価指標
    y_pred = model.predict(va_x)
    y_pred_list = y_pred.tolist()
    print(Counter(y_pred_list))

    print(f1_score(y_pred,va_y,average="macro"))
    f1_va_score.append(f1_score(y_pred,va_y,average="macro"))
    
    #testに対する確率の部分(y_prednumberに各ラベルの予測確立を収納)
    y_prob = model.predict_proba(test_x1).tolist() #19702,1
    y_prednumber += y_prob

    y_prob = np.argmax(y_prob,axis=1).tolist()
    print(Counter(y_prob))
    print("\n")
    

print(np.mean(f1_va_score))



Counter({1: 3924, 0: 1010, 2: 62})
0.3737791916318336
Counter({1: 15387, 0: 4058, 2: 257})


Counter({1: 3587, 0: 1124, 2: 285})
0.3725107098200952
Counter({1: 14086, 0: 4557, 2: 1059})


Counter({1: 3817, 0: 1044, 2: 135})
0.38598713146545843
Counter({1: 14941, 0: 4296, 2: 465})


Counter({1: 4009, 0: 847, 2: 140})
0.3856681260166048
Counter({1: 15487, 0: 3642, 2: 573})


0.379486289733498


In [114]:
y_test_prob = np.argmax(y_prednumber,axis=1).reshape([-1,1])
print(np.count_nonzero(y_test_prob == 0))
print(np.count_nonzero(y_test_prob == 1))
print(np.count_nonzero(y_test_prob == 2))
print(type(y_test_prob))
print(y_test_prob.shape)
print(y_test_prob.dtype)

index_test = pd.read_csv("../test.csv",index_col=0)
print(index_test.index.to_numpy())

3660
15428
614
<class 'numpy.ndarray'>
(19702, 1)
int64
[19984 19985 19986 ... 39966 39967 39968]


In [29]:
#optunaでパラメータ最適化
#こちらでバリデーションデータに対してスコアを
#最大化させてから実際に使うモデルのパラメータを決めました。
def objective(trial,train_x1,train_y):
    """
    max_depth = trial.suggest_int('max_depth', 1, 6)
    min_child_weight = trial.suggest_float('min_child_weight',1,5)
    gamma = trial.suggest_float('gamma',0.3,5)
    x = trial.suggest_float('x',3,6)
    y = trial.suggest_float('y',12,20)
    """
    reg_lambda = trial.suggest_float('reg_lambda',1,30)
    reg_alpha = trial.suggest_float('reg_alpha',0,5)
    
    kf = StratifiedKFold(n_splits=4,random_state=7777,shuffle=True)
    model = XGBClassifier(
        booster='gbtree',
        random_state=111,
        objective='multi:softprob',
        num_class=3,
        n_estimators=300,
        max_depth=2,
        learning_rate=0.1,
        eval_metric='aucpr',
        early_stopping_rounds=10,
        min_child_weight=2.265683041345576,
        gamma=3.939376582373192,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha
    )
    y_prednumber = np.zeros((len(test_x1),3)) #19702,4
    f1_va_score = []
    
    for tr_idx,va_idx in kf.split(train_x1,train_y):
        tr_x,va_x = train_x1.iloc[tr_idx],train_x1.iloc[va_idx]
        tr_y,va_y = train_y.iloc[tr_idx],train_y.iloc[va_idx]
        
        class_weights = compute_sample_weight({0:3.9553512894515057, 1:1,2:14.557580573218711}, y=tr_y)
        model.fit(tr_x,tr_y,sample_weight = class_weights,
                  eval_set=[[va_x,va_y]],
                  verbose=0)
        y_pred = model.predict(va_x)
        
        score = f1_score(y_pred,va_y,average="macro")
        f1_va_score.append(score)

    return np.mean(f1_va_score)


In [65]:
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial,train_x1,train_y), n_trials=1000)

[I 2023-12-31 00:17:54,623] A new study created in memory with name: no-name-e3dc7844-2866-4575-b5ff-89dd5f1dd651
[W 2023-12-31 00:17:55,210] Trial 0 failed with parameters: {'reg_lambda': 1.0235927612466513, 'reg_alpha': 0.7987857857185798} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\kirin\.virtualenvs\2023winter_SMBC_green_data_challenge-PlSiMd8v\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\kirin\AppData\Local\Temp\ipykernel_17200\3897654810.py", line 2, in <lambda>
    study.optimize(lambda trial: objective(trial,train_x1,train_y), n_trials=1000)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\kirin\AppData\Local\Temp\ipykernel_17200\2820246868.py", line 33, in objective
    tr_x,va_x = train_x1.iloc[tr_idx],train_x1.iloc[va_idx]
                ^^^^^^^^^^^^^
  File "C:\Users\kir

KeyboardInterrupt: 

In [115]:
index_list = index_test.index.to_numpy().reshape([-1,1])
print(type(index_list))

<class 'numpy.ndarray'>


In [116]:
data_numpy = np.hstack([index_list,y_test_prob])
print(data_numpy.shape)

(19702, 2)


In [117]:
np.savetxt('submission.csv',data_numpy,delimiter=',',fmt='%.0f')